In [1]:
### Package Import

import numpy as np
import pandas as pd

In [2]:
### Create detailed dataframes from the finals about what country gave how many points to what other country, from https://en.wikipedia.org/wiki/Eurovision_Song_Contest_20XX

years = list(range(2009,2024 + 1))
years.remove(2020) # There was no contest in 2020

def create_detailed_results_df_from_wiki(year):
    url = 'https://en.wikipedia.org/wiki/Eurovision_Song_Contest_' + str(year)
    tables = pd.read_html(url)
    t_prev = None
    for table in tables:
        if table.isin(['Total score']).any().any() or 'Total score' in table.columns:
            if year >= 2016: #from this year on, we have a different system of jury and public voting
                t_jury = t_prev
                t_prev = table
            t = table
    if year >= 2016:
        raw_tables = [t_jury,t]
    else:
        raw_tables = [t]
    for i in range(len(raw_tables)): 
        raw_tables[i] = raw_tables[i].drop(raw_tables[i].columns[0], axis = 'columns')
        for index, row in raw_tables[i].iterrows():
            if row.str.contains('Voting procedure used').any() and row.str.contains('Spain').any(): #Spain is just used here as some country that will be in every yeear
                raw_tables[i].columns = row
        raw_tables[i] = raw_tables[i].dropna(subset=[raw_tables[i].columns[0]])
        filt = raw_tables[i][raw_tables[i].columns[0]].str.contains('Voting procedure used')
        raw_tables[i] = raw_tables[i][~filt]
        raw_tables[i] = raw_tables[i].reset_index(drop = True)
        raw_tables[i] = raw_tables[i].fillna(0)
        raw_tables[i] = raw_tables[i].rename(columns = {raw_tables[i].columns[0]:'Country'})
        raw_tables[i] = raw_tables[i].set_index(raw_tables[i].columns[0])
        raw_tables[i] = raw_tables[i].map(lambda x: pd.to_numeric(x))
        raw_tables[i] = raw_tables[i].sort_values(by = 'Total score',ascending = False)
    if year >= 2016:
        globals()["df_" + str(year) + '_detailed_jury_prev'] = raw_tables[0]
        globals()["df_" + str(year) + '_detailed_public_prev'] = raw_tables[1]
    else:
        globals()["df_" + str(year) + '_detailed_prev'] = raw_tables[0]



for year in years:
    create_detailed_results_df_from_wiki(year)

In [3]:
### Create a dataframe with coordinates for every country, scraping from https://developers.google.com/public-data/docs/canonical/countries_csv

url = 'https://developers.google.com/public-data/docs/canonical/countries_csv'
countries_table = pd.read_html(url)[0]
countries_table = countries_table.set_index('name')
countries_table = countries_table.rename(index = {'Macedonia [FYROM]':'Macedonia'})
countries_table.loc['North Macedonia'] = countries_table.loc['Macedonia']
countries_table.loc['Czechia'] = countries_table.loc['Czech Republic']

In [4]:
### Create differently-structured dataframes who are best usable for the Tabelau map

for year in years:
    # Before year 2016, we only had one combined voting result. Starting from year 2016, we got two separate voting results - for jury and public.
    if year >= 2016:
        tables = [globals()["df_" + str(year) + '_detailed_jury_prev'], 
                  globals()["df_" + str(year) + '_detailed_public_prev']]
    else:
        tables = [globals()["df_" + str(year) + '_detailed_prev']]
    column_dropping_names = ['Total score', 'Jury score', 'Televoting score', 'Jury vote score', 'Rest of the World'] # Rest of the world is too ambiguous, so we drop it
    for i in range(len(tables)):
        dict_temp = {tables[i].index[j]:tables[i].iloc[j]['Total score'] for j in range(len(tables[i]))}
        for name in column_dropping_names:
            try:
                tables[i] = tables[i].drop([name], axis=1)
            except: 
                pass
        df = pd.DataFrame(columns = ['Finalist country', 
                                     'Giving country', 
                                     'Giving lat', 
                                     'Giving long', 
                                     'Finalist lat', 
                                     'Finalist long', 
                                     'Points given', 
                                     'Finalist Country - total points']
                         ).dropna(axis=1, how='all')
        for finalist_country in tables[i].index:
            for giving_country in pd.Series(tables[i].columns):
                new_row = pd.DataFrame({
                'Finalist country': [finalist_country], 
                'Giving country': [giving_country], 
                'Giving lat': [countries_table.loc[giving_country,'latitude']], 
                'Giving long': [countries_table.loc[giving_country,'longitude']], 
                'Finalist lat': [countries_table.loc[finalist_country,'latitude']], 
                'Finalist long': [countries_table.loc[finalist_country,'longitude']], 
                'Points given': [tables[i].loc[finalist_country,giving_country]],
                'Finalist Country - total points': dict_temp[finalist_country] 
                })
                df = pd.concat([df,new_row], ignore_index=True)
        tables[i] = df
        if year >= 2016:
            [globals()["df_" + str(year) + '_detailed_jury'], globals()["df_" + str(year) + '_detailed_public']] = tables
            globals()["df_" + str(year) + '_detailed_jury'].index.name = 'ID'
            globals()["df_" + str(year) + '_detailed_public'].index.name = 'ID'
        else:
            tables = [globals()["df_" + str(year) + '_detailed']] = tables
            globals()["df_" + str(year) + '_detailed'].index.name = 'ID'


In [5]:
### Creating Excel table with a sheet for every year

# Initiate the table with the first year. Make sure the first year is earlier than 2016
pd.DataFrame().to_excel('Eurovision_output_new.xlsx', sheet_name = str(years[0]))

for year in years:
    with pd.ExcelWriter('Eurovision_output_new.xlsx', mode = 'a', if_sheet_exists = 'overlay') as writer:
        if year < 2016:
            globals()["df_" + str(year) + '_detailed'].to_excel(writer,sheet_name = str(year))
        else:
            globals()["df_" + str(year) + '_detailed_jury'].to_excel(writer,sheet_name = str(year) + '_jury')
            globals()["df_" + str(year) + '_detailed_public'].to_excel(writer,sheet_name = str(year) + '_public')